In [124]:
from models import models
from datetime import datetime, timedelta
import time
from tortoise import Tortoise
from data import config
from tortoise.queryset import Q
import requests
from pprint import PrettyPrinter
import aiohttp
import asyncio
from models import models

pp = PrettyPrinter(indent=4)

In [146]:

async def check_transactions():
    await Tortoise.init(config=config.TORTOISE_ORM)
    params = {"address": "EQDcYqGh0d6i_kogT4QXvTuEq_iu61qPrZDVhbGnDvdrk-f3",
            "limit": 20}
    response = None
    for i in range(5):
        async with aiohttp.ClientSession() as session:
            async with session.get(url="https://toncenter.com/api/v2/getTransactions", params=params) as resp:
                if resp.status != 200:
                    await asyncio.sleep(1)
                    continue
                else:
                    response = await resp.json()
                    break
    if response is None:
        return
    if response['ok'] is True:
        transactions = response['result']
        pp.pprint(transactions[1])
        for transaction in transactions:
            if transaction['transaction_id']['hash'] == "KfOUWLY6Hj80BjW533q8WOeABF1JumhlRU1Ua32ABJo=":
                return
            if transaction['in_msg']['value'] == '0':
                continue
            
            amount = int(transaction['in_msg']['value']) / 1000000000
            hash = transaction['transaction_id']['hash']
            wallet = transaction['in_msg']['source']
            code = transaction['in_msg']['message']

            existing_transaction = await models.UserBalanceChange.get_or_none(hash=hash)
            if existing_transaction:
                continue

            user_balance_change = None
            if code != '':
                user_balance_change = await models.UserBalanceChange.get_or_none(code=code, state="created")
            
            if user_balance_change is not None:
                user = await user_balance_change.user
                user_balance_change.amount = amount
                user_balance_change.state = "done"
                user_balance_change.hash = hash
                user_balance_change.wallet = wallet
                user.balance += amount
                await user_balance_change.save()
                await user.save()
            else:
                await models.UserBalanceChange.create(user=None,
                                                     type="topup",
                                                     amount=amount,
                                                     hash=hash,
                                                     wallet=wallet,
                                                     code=code,
                                                     state="done")

await check_transactions()

{   '@type': 'raw.transaction',
    'data': 'te6cckECBQEAASkAA7V9xioaHR3qL+SiBPhBe9O4Sr+K7rWo+tkNWFsacO92uTAAAak+ruSwPnoqCOI3YO2MOQebk0bEs7tM49Tq6v3ZvgqatJe5JTMgAAGlQ+6IHBYr+aPwABRgbcDIAQIDAQGgBACCcjw9zIrwMBBDF81x8ilTIXeEMUCMdXBIdlGADHM6Z4C7fcjuEdRE4RKA1X/rJP3LnTu4iN/I+izbmzQRb7Pw2LYADwzA24GIQokgAOtIADZCas5UJjh6zRVZjOn563lOeSbLkpl/PVgmAv29cIipADcYqGh0d6i/kogT4QXvTuEq/iu61qPrZDVhbGnDvdrkxCgGFFhgAAA1J9XclgTFfzR+AAAAACC0uTI5N7gQFpAwtLkyOTe4Fro3txc3uTOQEZjALEVPNg==',
    'fee': '224774',
    'in_msg': {   '@type': 'raw.message',
                  'body_hash': '29ePvvvLKix+Z0VOawmJsOm+nq53TeCnxH0c6DR45ys=',
                  'created_lt': '29222604000002',
                  'destination': 'EQDcYqGh0d6i_kogT4QXvTuEq_iu61qPrZDVhbGnDvdrk-f3',
                  'fwd_fee': '666672',
                  'ihr_fee': '0',
                  'message': 'Airdrop - airdrop-ton.org #1',
                  'msg_data': {   '@type': 'msg.dataText',
                                  'text': 'QWlyZHJvcCAtIGFpcmR

In [34]:
1000000 / 1000000000

0.001